<a href="https://colab.research.google.com/github/adventHymnals/resources/blob/master/adventHymnals_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section 1
You'll need to expand to interact with interactive cells.

Installation of pdflatext takes quite a while

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.environ['ROOT_DIR']="adventhymnals_v4"
os.environ['PRIVATE_DIR']="private"
os.environ['hymnals_repo']="v4"

import getpass
password = getpass.getpass("Please enter ssh zip password: ")
os.environ['SSH_ZIP_PASSWORD'] = password

In [ ]:
#%%capture
import os
####
#### sheets
####
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

file_path = os.environ['ROOT_DIR'] + '/public/Hymnals'
print(file_path)

#worksheet = gc.open(file_path, folder_id="1uXjDtbXRe78EhnCzOe8gjk-gXOdyPNsh").sheet1
worksheet = gc.open("Hymnals").sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
#import pandas as pd
#pd.DataFrame.from_records(rows)

hymnals = [row[0] + '-v4' for row in rows[1:]] ## how will we get the rest of the fields

# Convert to YAML format
yaml_string = "hymnals:\n"
for hymnal in hymnals:
    yaml_string += f"  - '{hymnal}'\n"

#print(yaml_string)

# Save to a file named 'hymnals.yaml'
file_path = '/tmp/hymnals.yaml'
with open(file_path, 'w') as file:
    file.write(yaml_string)

print(f"YAML file saved as '{file_path}'")


### process configs
worksheet = gc.open("configs").sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# print(rows)
for row in rows:
    key, value = row
    os.environ[key] = value


####
#### Convert the Rows to JSON
####
import json

worksheet = gc.open("Hymnals").sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

def rows_to_json(rows):
    if len(rows) < 2:
        return "Error: Not enough rows to create JSON"

    json_data = {}

    for row in rows[1:]:
        key = row[0]
        values = row[1:]
        json_data[key] = {}

        header = rows[0][1:]  # Exclude the first item (key) from the header
        for i, value in enumerate(values):
            json_data[key][header[i].lower()] = value

    #return json.dumps(json_data, indent=4)
    json_filename = "/tmp/hymnalsdata.json"
    with open(json_filename, 'w') as file:
        json.dump(json_data, file, indent=4)
#print(rows)
json_output = rows_to_json(rows)
#print(json_output)

In [ ]:
%%shell
apt install jq

In [ ]:
%%shell
#apt install jq
extract_values_from_json() {
    filename="$1"
    key1=$2 #(echo "$2" | tr '[:upper:]' '[:lower:]') # don't convert this one to lowercase
    key2=$(echo "$3" | tr '[:upper:]' '[:lower:]')

    # Check if jq is available
    if ! command -v jq &>/dev/null; then
        echo "Error: jq is not installed. Please install jq to use this function."
        return 1
    fi

    # Check if the file exists
    if [ ! -f "$filename" ]; then
        echo "Error: File '$filename' not found."
        return 1
    fi
    # Use jq to extract the values for the given keys
    value1=$(jq -r --arg key "$key1" '.[$key]' "$filename")
    value2=$(echo "$value1" | jq -r --arg key "$key2" '.[$key]')
    echo "$value2"
}
#extract_values_from_json $1 $2 $3
# Save the function into a file named "extract_json_values.sh"
cat <<EOF > /tmp/extract_json_values.sh
#!/bin/bash

$(declare -f extract_values_from_json)

# Main script logic
# Add any additional code here if needed
#extract_values_from_json "$1" "$2" "$3"
EOF

### problem with the function, save the
### save instead the
sed -i 's/extract_values_from_json ()//' /tmp/extract_json_values.sh
sed -i 's/[{}]//' /tmp/extract_json_values.sh
#sed -i 's/()//' /tmp/extract_json_values.sh
#cat /tmp/extract_json_values.sh
# Make the file executable
chmod +x /tmp/extract_json_values.sh
#/tmp/extract_json_values.sh "/tmp/hymnalsdata.json" campus-melodies "TITLE"
#a=$(source /tmp/extract_json_values.sh "/tmp/hymnalsdata.json" campus-melodies "TITLE")
#echo $a

In [ ]:
with open('/tmp/my_script.py', 'w') as file:
    file.write('''\
import sys
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

file_path =  sys.argv[1] + 'public/Hymnals' #os.environ['ROOT_DIR'] +
#print(file_path)

worksheet = gc.open("Hymnals").sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

data = rows
hymnal_to_find = sys.argv[3].lower()  # Convert the hymnal to lowercase (case insensitive)
search_field = sys.argv[2].lower()  # Convert the search field to lowercase

found = False
row_number=-1
for index, field in enumerate(data[0]):
    if field.lower() == search_field:  # Convert the field to lowercase for case-insensitive comparison
        #print(f"The index of '{search_field}' is: {index}")
        found = True
        row_number = index
        break

if not found:
    #print(f"'{search_field}' not found in the list.")
    sys.exit(1)

column_number = -1
for idx, row in enumerate(data):
    if row[0].lower() == hymnal_to_find:
        column_number = idx
        break

if column_number == -1:
    #print(f"Error: '{sys.argv[3]}' not found in the first items of any row==.")
    sys.exit(1)

#print(column_number)

result_value = data[column_number][row_number]
print(result_value)
#print(f"The value at row  and column '{sys.argv[3]}' is: {result_value}")


''')
#!echo $(python /tmp/my_script.py $ROOT_DIR "TITLE" "campus-melodies")

In [ ]:
%%shell

sshZipped="/content/drive/MyDrive/$ROOT_DIR/$PRIVATE_DIR/ssh.zip"
cp $sshZipped /tmp/
unzip -P "$SSH_ZIP_PASSWORD" -oq /tmp/ssh.zip
rm -rf /root/.ssh
mkdir -p /root/.ssh
rsync -aq ssh/  ~/.ssh/
rsync -aq ssh/   /root/.ssh/
ssh-keyscan GitHub.com > /root/.ssh/known_hosts #2>&1 >/dev/null
ssh-keyscan GitHub.com > ~/.ssh/known_hosts #2>&1 >/dev/null
chmod 644 ~/.ssh/known_hosts
chmod 600 ~/.ssh/id_rsa
chmod 644 /root/.ssh/known_hosts
chmod 600 /root/.ssh/id_rsa

#eval "$(ssh-agent)"
#ssh-add ~/.ssh/id_rsa
#echo "$SSH_ZIP_PASSWORD" | ssh-add ~/.ssh/id_rsa

eval "$(ssh-agent)"
echo "$SSH_ZIP_PASSWORD" | ssh-add ~/.ssh/id_rsa
#rm -rf $hymnals_repo
rm -rf $hymnals_repo
git clone "git@github.com:adventHymnals/$hymnals_repo.git"
mv "/tmp/hymnals.yaml" "$hymnals_repo"

echo "# Hymnals" > /tmp/tmpIndex.md
echo " " >> /tmp/tmpIndex.md
echo "|All Hymnals|" >> /tmp/tmpIndex.md

#
cd $hymnals_repo
cat hymnals.yaml
cat hymnals.yaml | grep -o "'[^\']*'" | sed "s/'//g" | while read -r hymnal; do
  hymnal_v4=$hymnal
  hymnal=$(echo "$hymnal"|sed -e 's/\-v4$//')
  echo "===>$hymnal"
  #title=$(python /tmp/my_script.py $ROOT_DIR "TITLE" "$hymnal")
  available=$(/tmp/extract_json_values.sh "/tmp/hymnalsdata.json" $hymnal "Available")
  if [ "$available" != "TRUE" ]; then  ## proceed only if hymnal is set to available
      continue
  fi
  echo $available
  title=$(/tmp/extract_json_values.sh "/tmp/hymnalsdata.json" $hymnal "TITLE")
  #echo -n $hymnal
  #echo "|[$title](/$hymnal_v4)|"
  echo "|[$title](/$hymnal_v4)|" >> /tmp/tmpIndex.md
done

cd ..

git config --global user.email "$GitCommitEmail"
git config --global user.name "$GitCommitUser"
cd $hymnals_repo
mv /tmp/tmpIndex.md index.md
# echo "$(date)" >> tmp
git add . && git commit -m "$(date)" && git push origin master
cd ..


cd $hymnals_repo
eval "$(ssh-agent)"
echo "$SSH_ZIP_PASSWORD" | ssh-add ~/.ssh/id_rsa
#rm -rf $hymnals_repo
rm -rf tmps
mkdir -p tmps

cat hymnals.yaml | grep -o "'[^\']*'" | sed "s/'//g" | while read -r hymnal; do
    echo "=>$hymnal"
    csycms_hymnal=$(echo "$hymnal"|sed -e 's/v4$/csycms/')
    #echo $csycms_hymnal

    cd tmps
    git clone "git@github.com:adventHymnals/$hymnal.git"
    ls "$hymnal"
    cd "$hymnal"

    if git rev-parse HEAD >/dev/null 2>&1; then
      echo "Repository is not empty."
    else ## if _v4 repo does not exist
      currentPath=$(pwd)
      cd /tmp
      rm -rf "$csycms_hymnal"
      if git clone "git@github.com:adventHymnals/$csycms_hymnal.git"; then

        cd "$csycms_hymnal"
        rm -rf .git .github

        # rename /chapter.md
        find "./" -wholename "*/chapter.md"  | while read -r line ; do
          newPath=$( echo $line | sed -e 's|/chapter\.md|/index.md|' )
          mv "$line" "$newPath"
        done

        # rename /docs.md
        find "./" -wholename "*/docs.md"  | while read -r line ; do  ## remove also trailing comma from file names
            newPath=$( echo $line | sed -e 's|/\([^\]*\)/docs\.md|/\1.md|' | sed -e 's|,\.md|.md|')
            # echo "$line==>$newPath"
            mv "$line" "$newPath"
        done

        if [ -d "./indices" ]; then # if its a hymnal, (with indices dir)
            mv "./index.md" "$hymnal/preface.md"
            mv "./indices/index.md" "$hymnal/index.md"
            rm -rf "./indices"
        fi

        find  "./" -type d -empty -delete
        #ls -lha ./
        cd ..
        rsync -av "$csycms_hymnal/" "$currentPath/"
        cd "$currentPath"
        git add . && git commit -m "$(date)" && git push origin master
        #ls -lha
      fi
    fi
    cd ..
    cd ..
done

Create the _v4 hymnal repos
No means of filtering spreadsheets by directory, so it is important that spreadsheets don't share titles
https://docs.gspread.org/en/latest/api/client.html#gspread.Client.open

In [ ]:
#%%capture
%%shell
cd /content/$hymnals_repo
cd tmps

for hymnal in *v4*; do
    echo "=>$hymnal"
    #lastGitTime=$(git log -1 --format="%at")
    #echo "Unix Timestamp: $lastGitTime"
    #break
    drive_hymnal=$(echo "$hymnal"|sed -e 's/-v4$//')
    #echo "<==$csycms_hymnal"
    if [ -d "/content/drive/MyDrive/$ROOT_DIR/public/Hymnals/$drive_hymnal" ]; then
      echo "Folder exists!"
    else
      echo "Folder does not exist!"
      #// do the conversion here...


      source_folder="./$hymnal"  # Replace with the path to the source folder
      output_folder="/content/drive/MyDrive/$ROOT_DIR/public/Hymnals/$drive_hymnal"  # Replace with the path to the output folder

      # Create the output folder if it doesn't exist
      mkdir -p "$output_folder"

      # Function to convert .md files to .docx
      convert_to_docx() {
        local source_file="$1"
        local output_file="$2"
        pandoc "$source_file" -o "$output_file"
      }

      # Find all .md files in the source folder and its subfolders
      find "$source_folder" -type f -name "*.md" |
      while IFS= read -r md_file; do
      relative_path="${md_file#$source_folder/}"
        #echo $relative_path
        # Determine the output file path by replacing the .md extension with .docx in the output folder
        docx_file="${md_file%.md}.docx"
        output_file="$output_folder/${relative_path%.md}.docx"
        #output_file="$output_folder/${docx_file##*/}"
        mkdir -p "$(dirname "$output_file")"
        echo "==>$output_file"

        # Convert .md to .docx and save it in the output folder
        file_name="$md_file"
        #cat $file_name
        sed -i '/^#/d' $file_name
        ## Format the title. Remove hymn number and hymnal name
        sed -i 's/^\([ ]*\)[0-9]*[\.]*[ ]*\(.*\) \- [^\-]*$/\1\2/' $file_name
        sed -i 's/^\-\-$/---/g' $file_name
        sed -i '/^\^/d' $file_name
        ## sed -i '/```/{:a;N;/```/!ba;s/```.*$/```/2;}' $file_name
        new_file_name="new_file_name"
        ## Remove everything after the last ```
        #cat $file_name
        grep -n -m 2 '```' $file_name | tail -n 1 | cut -d ':' -f 1 | xargs -I {} head -n {} $file_name > "$new_file_name"
        #sed -i 's/\(```.*```\).*/\1/' $file_name
        #cat $new_file_name
        mv $new_file_name $file_name

       convert_to_docx "$md_file" "$output_file"
       #cat $new_file_name
      done

    fi
done

Sync github repo with our docs here.
If folder does not exist in drive, run pandoc to convert every file to docx and save in drive, otherwise

Run this cell atleast 3 times until the only output is of the hymnal dir names. If an error occurs, you will have to rename the directory causing the error manually

In [ ]:
%%shell

cd /content/$hymnals_repo
cd tmps

echo "" > /tmp/modifiedHymnals
# if matched..
for hymnal in *; do

    source_folder="./$hymnal"  # Replace with the path to the source folder
    drive_hymnal=$(echo "$hymnal"|sed -e 's/-v4$//')
    output_folder="/content/drive/MyDrive/$ROOT_DIR/public/Hymnals/$drive_hymnal"  # Replace with the path to the output folder

    #echo "<==$csycms_hymnal"
    lastGitTime=$(git log -1 --format="%at")
    #echo "Unix Timestamp: $lastGitTime"

    #last_modified=$(stat -c "%Y" "$output_folder") # or date -r $output_folder +"%s"
    readarray -d '' files < <(find "$output_folder" -type f -print0)

    # Initialize variables to hold information about the last modified file
    last_modified_file=""
    last_modified_time=0

    # Loop through the list of files to find the last modified file
    for file in "${files[@]}"; do
        # Get the last modification time of the current file using stat
        current_modified_time=$(stat -c "%Y" "$file")

        # Compare the last modification time of the current file with the current maximum
        if ((current_modified_time > last_modified_time)); then
            last_modified_time=$current_modified_time
            last_modified_file=$file
        fi
    done
    #stat -c "%y" "$last_modified_file"
    last_modified=$(stat -c "%Y" "$last_modified_file")
    #echo "Last modified: $last_modified"
    if ((lastGitTime > last_modified)); then  ## nothing has changed
      #echo $lastGitTime
      #echo $last_modified
      #echo "Nothing has changed"
      continue
    fi
    echo "=>$drive_hymnal"
    echo $drive_hymnal >> /tmp/modifiedHymnals
    #continue


      # Find all .md files in the source folder and its subfolders
      find "$output_folder" -type f -name "*.docx" |
      while IFS= read -r file_path; do

        filename=$(basename "$file_path")
        pattern="^[0-9]{2,3}\..*\.docx"
        if [[ $filename =~ $pattern ]]; then
            folder_name=$(dirname "$file_path")
            folder_number=$(echo "$folder_name" | sed 's/.*\.\([0-9]*\).*/\1/' | sed 's/^[0]*//' )
            file_number=$(echo "$file_path" | sed 's/.*\/\([0-9]*\)\..*/\1/' | sed 's/^[0]*//')
            result=$((folder_number + file_number - 1))
            fileNumber=$(printf "%04d" "$result")
            if [ "$fileNumber" = "0000" ]; then
              echo "Variable is 0000."
              fileNumber=""
              newFileName=$(echo "$filename" | sed 's/\([0-9]*\)\(\..*\)/\2/' |sed 's/\.//')
            else
              newFileName=$(echo "$filename" | sed 's/\([0-9]*\)\(\..*\)/\2/')
            fi

            #newFileName=$(echo "$filename" | sed 's/\([0-9]*\)\(\..*\)/\2/')
            newFileName="$fileNumber$newFileName"
            newFileName="$folder_name/$newFileName"
            echo "=>$newFileName"
            mv "$file_path" "$newFileName"
            # echo "$newFileName"
        #else
            #echo "Filename does not match the pattern."
        fi
      done


      find "$output_folder" -type f -name "*.docx" |
      while IFS= read -r file_path; do

        filename=$(basename "$file_path")
        folder_name=$(dirname "$file_path")
        #tmp=$(dirname $tmp)
        #echo $tmp | sed 's/[0-9]*\.\([^\.]*\.[^\.]*$\)/\1/'
        newDir=$(echo $folder_name | sed 's/[0-9]*\.\([^\.]*$\)/\1/') # move last dir, then go down the chain
        #echo "$folder_name=>$newDir"
        #mv "$folder_name" "$newDir"
        if [ -d "$folder_name" ] && [ "$folder_name" != "$newDir" ]; then
          #echo "Directory '$directory_name' exists."
          echo "$folder_name=>$newDir"
          mv "$folder_name" "$newDir"
        #else
          #echo "Directory '$directory_name' does not exist."
        fi
      done
done

# move...

## Section 2

In [ ]:
%%shell
apt-get install xattr > /dev/null
sudo apt-get install texlive-latex-base
sudo apt-get install texlive-fonts-recommended
sudo apt-get install texlive-fonts-extra

Only convert files which have been modified. Delete files which have been removed.

In [ ]:
%%shell
wget https://raw.githubusercontent.com/dialoa/columns/master/columns.lua -O /tmp/columns.lua
eval "$(ssh-agent)"
echo "$SSH_ZIP_PASSWORD" | ssh-add ~/.ssh/id_rsa
cd /content/$hymnals_repo
cd tmps

# if matched..
for hymnal in *; do
    drive_hymnal=$(echo "$hymnal"|sed -e 's/-v4$//')
    if grep -qF "$drive_hymnal" "/tmp/modifiedHymnals"; then
        echo ""
    else
        #echo "Missing: $drive_hymnal."
        continue # do nothing with directories that have not been modified
    fi
    echo "=>$hymnal"
    lastGitTime=$(git log -1 --format="%at")
    echo $lastGitTime
    #ls -lha "/tmp/$drive_hymnal/$drive_hymnal"
    pwd
    rm -rf "/tmp/$hymnal-gh"
    cp -r $hymnal "/tmp/$hymnal-gh"
    currentDir=$(pwd)
    cd "/tmp/$hymnal-gh"
    git branch gh-pages
    git checkout gh-pages
    git pull origin gh-pages
    ls -lha ./
    cd $currentDir
    #continue
    #continue

    #echo "<==$csycms_hymnal"



      source_folder="./$hymnal"  # Replace with the path to the source folder
      output_folder="/content/drive/MyDrive/$ROOT_DIR/public/Hymnals/$drive_hymnal"  # Replace with the path to the output folder

      rm -rf "/tmp/$drive_hymnal"
      mkdir -p "/tmp/$drive_hymnal"
      echo "" > "$output_folder/fullhymnal.md"

      echo "[Download](/$hymnal/$hymnal.pdf)" > /tmp/index2
      echo " " >> /tmp/index2
      echo "[Full Hymnal](/$hymnal/$hymnal.html)" >> /tmp/index2
      echo " " >> /tmp/index2
      echo "|Songs|" >> /tmp/index2
      echo "|------|" >> /tmp/index2

     # find "$output_folder" -type f -name "*.docx" | sort
     # break
     echo "" > /tmp/existingFiles

      find "$output_folder" -type f -name "*.docx" | sort |
      while IFS= read -r file_path; do

        filename=$(basename "$file_path")
        #echo $file_path
        pattern="^[0-9]{4}\..*\.docx"
        if [[ $filename =~ $pattern ]]; then
            tmpPath=$(echo $file_path | sed 's/.*Hymnals\///' |sed 's/\.docx/.md/')
            tmpDir=$(dirname $tmpPath)
            tmpPathtmpPath=$(basename $tmpPath | sed 's/^[0-9]*\.//' | sed 's/://g')
            #echo $tmpPath
            #echo $tmpDir

            function add_one_to_number_before_md() {
                local str="$1"
                if grep -qxF "$str" "/tmp/existingFiles"; then
                    if [[ $str =~ ([0-9]+)(\.md)$ ]]; then
                        number=${BASH_REMATCH[1]}
                        ((number++))
                        str="${str//${BASH_REMATCH[1]}/${number}}"
                        str=$(add_one_to_number_before_md "$str")
                    else
                        str=$(echo "$str" | sed "s/\(.*\)\.md$/\1-1.md/")
                        str=$(add_one_to_number_before_md "$str")
                    fi
                fi
                echo "$str"
            }

            tmpPathtmpPath=$(add_one_to_number_before_md "$tmpPathtmpPath")

            #echo "Result: $tmpPathtmpPath"
            echo "$tmpPathtmpPath" >> /tmp/existingFiles
            #echo $tmpPathtmpPath
            #continue
            file_modified_time=$(stat -c "%Y" "$file_path")
            #echo $file_modified_time

            #break
            if ((file_modified_time > lastGitTime)); then ## file has been modified
                fid=$(xattr -p 'user.drive.id' "$file_path")
                fid=$(echo "$fid" | sed 's/-/%2D/g')

                pandoc "$file_path" -s -o "/tmp/$drive_hymnal/tmp.md"
                file_number=$(echo "$filename" | sed 's/\([0-9]*\)\..*/\1/' | sed 's/^[0]*//')
                #echo $file_number

                ## sed -i "s/title:[ ]*'\(.*\)',/## \1/g" "$output_folder/fullhymnal.md"
                # remove ' if both first and last chars are ', eg in: ‘Delightful day! first gift of Heaven’
                sed -i "s/title:\(.*\)/## $file_number. \1\n[edit](https:\/\/docs.google.com\/document\/d\/$fid\/edit?mode=html)\n/g" "/tmp/$drive_hymnal/tmp.md"
                sed -i "s/\-\-\-//g" "/tmp/$drive_hymnal/tmp.md"
                #sed -i '1s/\(\[\]{#.*}\)//' "/tmp/$drive_hymnal/tmp.md"
            else

                #fid=$(xattr -p 'user.drive.id' "$file_path")
                #fid=$(echo "$fid" | sed 's/-/%2D/g')

                #pandoc $file_path -s -o "/tmp/$drive_hymnal/tmp.md"
                #file_number=$(echo "$filename" | sed 's/\([0-9]*\)\..*/\1/' | sed 's/^[0]*//')
                #echo $file_number

                #### sed -i "s/title:[ ]*'\(.*\)',/## \1/g" "$output_folder/fullhymnal.md"
                ### remove ' if both first and last chars are ', eg in: ‘Delightful day! first gift of Heaven’
                #sed -i "s/title:\(.*\)/## $file_number. \1\n[edit](https:\/\/docs.google.com\/document\/d\/$fid\/edit?mode=html)\n/g" "/tmp/$drive_hymnal/tmp.md"
                #sed  -i "s/\-\-\-//g" "/tmp/$drive_hymnal/tmp.md"



                existingMdPath=$(echo $tmpPath | sed 's/[^\/]*\///')
                existingMdDir=$(dirname $existingMdPath)
                existingMdDirFileName=$(basename $existingMdPath | sed 's/^[0-9]*\.//' | sed 's/://g')
                #existingMdPath="$existingMdDir/$existingMdDirFileName"
                existingMdPath="$existingMdDir/$tmpPathtmpPath"
                cat "/tmp/$hymnal-gh/$existingMdPath" > "/tmp/$drive_hymnal/tmp.md"
                ##cat "/tmp/$drive_hymnal/tmp.md"


            fi
            ###### check date of file path, compare with epoch. Because we are also doing full hymnal...
                ## file id

            # Remove quotes from titles
            sed -i "s/\(## [0-9]*\. *\)'/\1/" "/tmp/$drive_hymnal/tmp.md" # remove leading ' from title
            #sed -i "s/  */ /g" "/tmp/$drive_hymnal/tmp.md" Remove double spaces
            #sed -i "s/\(## [0-9]*\..*\)'$/\1/" "/tmp/$drive_hymnal/tmp.md"
            sed -i '/./,$!d' "/tmp/$drive_hymnal/tmp.md" # delete all leading blank lines from the file
            sed -i "1 s/'$//" "/tmp/$drive_hymnal/tmp.md" # remove last ' in first line
            sed -i '1s/\(\[\]{#.*}\)//' "/tmp/$drive_hymnal/tmp.md" ## Remove the anchor
            #sed -i '2 s/,$//' "/tmp/$drive_hymnal/tmp.md"
            sed -i "s/''*/'/g" "/tmp/$drive_hymnal/tmp.md"
            # remove double spaces
            sed -i "s/\([^ ]\)  *\([^ ]\)/\1 \2/g" "/tmp/$drive_hymnal/tmp.md"
            sed -i "s/  *$//g" "/tmp/$drive_hymnal/tmp.md"
            #sed -i "s/\^( *[^ ]\)\([^ ]*\)  /\1\2 /g" "/tmp/$drive_hymnal/tmp.md"
            #sed -i -E 's/(^|[^ ])  /\1 /g' "/tmp/$drive_hymnal/tmp.md"
            #sed -i 's/^[ \t]*//' "/tmp/$drive_hymnal/tmp.md"
            ## Use awk to process the file and replace line breaks as required
            #file="/tmp/$drive_hymnal/tmp.md"
            #awk 'NR > 2 { if (sub(/\\$/, "")) { printf "%s", $0 } else { printf "%s\\", $0 } } NR > 2 && NR < 3 { printf "%s", $0 } NR == 2 { print $0 }' "$file" > temp.txt && mv temp.txt "$file"

           ## cat "/tmp/$drive_hymnal/tmp.md" >> "$output_folder/fullhymnal.md"
           ##echo " " >> "$output_folder/fullhymnal.md"
           ## echo " " >> "$output_folder/fullhymnal.md"


            #rm -rf "/tmp/$drive_hymnal/$tmpdrive_hymnalDir"
            mkdir -p "/tmp/$drive_hymnal/$tmpDir"

            cat "/tmp/$drive_hymnal/tmp.md" >> "/tmp/$drive_hymnal/$tmpDir/$tmpPathtmpPath"
            tmpPath="$tmpDir/$tmpPathtmpPath"

            # Pattern to check for
            pattern="^\(([a-z])\)"
            ## add a, b etc if more than single song of number
            numberAndTitle=$(grep -o '##.*' "/tmp/$drive_hymnal/$tmpPath" | sed "s/\(## [0-9]*\. *\)'/\1/" | sed "s/  */ /g" | sed "s/\(## [0-9]*\..*\)'$/\1/" | sed "s/''*/'/g"| sed 's/## *//')
            if [[ "${tmpPathtmpPath}" =~ $pattern ]]; then
              matched_pattern="${BASH_REMATCH[1]}"
              #echo "The variable starts with the pattern '(${matched_pattern})'. $numberAndTitle"
              numberAndTitle=$(echo $numberAndTitle|sed "s/\([0-9]*\)\./\1${matched_pattern}./")
              sed -i "1s/.*/## $numberAndTitle/" "/tmp/$drive_hymnal/$tmpDir/$tmpPathtmpPath"
            #else
              #echo "The variable does not start with the specified pattern."
            fi
            cat "/tmp/$drive_hymnal/$tmpDir/$tmpPathtmpPath" >> "$output_folder/fullhymnal.md"
            echo " " >> "$output_folder/fullhymnal.md"
            echo " " >> "$output_folder/fullhymnal.md"
           #echo $matched_pattern
           echo $numberAndTitle
           #continue
            hrefPath_=$(echo $file_path | sed 's/.*Hymnals\///' |sed 's/\.docx/.html/' | sed 's|^[^/]*/||') #| sed 's|^[^/]*/||'
            hrefPath=$(dirname $hrefPath_)
            hrefFile=$(basename  $hrefPath_| sed 's/^[0-9]*\.//')
            #echo $hrefPath
            #hrefPath="/$hymnal/$hrefPath/$hrefFile"
            hrefPath="/$hymnal/$hrefPath/$tmpPathtmpPath"
            #echo "$hrefPath"
            echo "|[$numberAndTitle]($hrefPath)|" >> /tmp/index2
        fi
      done
      rm "/tmp/$drive_hymnal/tmp.md"
      echo "" >> /tmp/index2
      cp "$output_folder/fullhymnal.md" "/tmp/$drive_hymnal/$drive_hymnal/$hymnal.md"
      cat /tmp/index2 > "/tmp/$drive_hymnal/$drive_hymnal/index.md"
      #cat "$output_folder/fullhymnal.md" >> "/tmp/$drive_hymnal/tmp.md"
      #cat "/tmp/$drive_hymnal/tmp.md" > "$output_folder/fullhymnal.md"
      #rm "/tmp/$drive_hymnal/tmp.md"
      ## columns: https://levelup.gitconnected.com/use-columns-adjust-margins-and-do-more-in-markdown-with-these-simple-pandoc-commands-adb4c19f9f35
      cat "$output_folder/fullhymnal.md" > /tmp/fullhymnal
      echo "::: columns" > "$output_folder/fullhymnal.md"
      cat /tmp/fullhymnal >> "$output_folder/fullhymnal.md"
      echo ":::" >> "$output_folder/fullhymnal.md"
      pandoc "$output_folder/fullhymnal.md" -o "$output_folder/fullhymnal.docx"
      #pandoc --toc "$output_folder/fullhymnal.docx" -o "$output_folder/fullhymnal.pdf"
      pandoc  "$output_folder/fullhymnal.docx" -o "$output_folder/fullhymnal.pdf"
      cp "$output_folder/fullhymnal.pdf" "/tmp/$drive_hymnal/$drive_hymnal/$hymnal.pdf"

      ## try columns
      #pandoc --toc "$output_folder/fullhymnal.md" --lua-filter=/tmp/columns.lua -o "$output_folder/fullhymnal.pdf"
      pandoc "$output_folder/fullhymnal.md" --lua-filter=/tmp/columns.lua -o "$output_folder/fullhymnal.pdf"
      cp "$output_folder/fullhymnal.pdf" "/tmp/$drive_hymnal/$drive_hymnal/fullhymnal.pdf"
      ## E O Columns
      #mv "/tmp/$drive_hymnal/$tmpDir/" "/tmp/$drive_hymnal/$tmpDir-bac/"
      cd "./$hymnal"
      cp -r .git "/tmp/$drive_hymnal/$drive_hymnal/"
      cd ..
      currentDir=$(pwd)
      cd "/tmp/$drive_hymnal/$drive_hymnal/"
      git branch gh-pages ## inscase of an error, don't destroy master branch of main repo
      git checkout gh-pages
      #find "./" -type d -not -name '.*' -exec rm -r {} +
      #rsync -av "/tmp/$drive_hymnal/$drive_hymnal/" "./"  ## For some reason "/tmp/$drive_hymnal/$tmpDir/*" with the trailing slash alone
      #touch .nojekyll
      git add . && git commit -m "$(date)" && git push origin gh-pages -f
      #cd ..
      cd "$currentDir"
done

# move...

# Saving data to Google Sheets


This snippet uses the open-source [gspread](https://github.com/burnash/gspread) library for interacting with Sheets.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.create('A new spreadsheet')

# Open our new sheet and add some data.
worksheet = gc.open('A new spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)
# Go to https://sheets.google.com to see your new spreadsheet.